# <center> **Compte-rendu projet Optimisation**
<div align = right> Camille Dubois & Florian Garbe

On pose la fonction suivante à minimiser :
$$
\begin{align*}
 f: p= \left(\begin{matrix} p_1 \\ \vdots\\ p_N \end{matrix}\right) \longrightarrow \sum_{i=0}^{N-1} \|p_{i+1}-R_i\Delta p_{i,i+1} -p_i \|_2^2 + \gamma \sum_{i, j \in L}\|p_j - R_i\Delta p_{i,j}-p_i\|_2^2
 \\
 c_1(p) = p_{,3} - \pi = \theta_i - \pi \leq 0 \\
 c_2(p) = p_{,3} + \pi = \theta_i + \pi \leq 0
 \end{align*}
$$



Ainsi les variables de décisions sont $p = \left(\begin{matrix} p_1 \\ \vdots \\ p_N\end{matrix}\right) \in \mathbb{M}_{3, N} $ au nombre de $3N$.

---

---

### Question 3.
&ensp;&ensp;&ensp; On réalise la fonction permettant de lire les données du fichier INTEL.txt qui contient 1227 lignes ODO, donc 3 681 incconues sont liées à ce jeu de données.

In [5]:
# on importe les modules nécessaires
import numpy as np

In [ ]:
def lec_don(file:str):
    """ Cette fonction retourne les deltap_i,i+1 associé au mesures d'odométrie
        ainsi que les deltap_i,j associé au mesures de fermetures
        
        return a tuple of array. 
            - The first array is the deltap_i,i+1 linked to the odométrie sorted
            in rising order.
            
            - The second one is linked to the closing sorted as well in rising
            order.
            
        parameters :
            - The file that contains the data"""

    ODO = np.zeros((1227, 3)) # l'array d'odométrie
    LOOP = np.zeros((200, 5)) # L'array de fermeture
    num_line = 0 # Le numéro de la ligne en cours

    with open file as file:
        for line in file:
            type, p1, p2, dx, dy, dtheta = line.split(' ')
            if type == "EDGE_ODO":
                ODO[num_line] = [float(dx), float(dy), float(dtheta)]
            if type == "EDGE_LOOP":
                LOOP[num_line] = [int(p1), int(p2), float(dx), float(dy), float(dtheta)]
            
            num_line +=1

    return (ODO, LOOP)



In [7]:
np.zeros((2,1))

array([[0.],
       [0.]])